# Optimization

## Import Libraries

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import importlib
import numpy as np
import pandas as pd

from src import config
import src.optimization_utils as ou
import src.solutions as s
import src.display as disp
import src.runs as runs
from src.problem import SpineProblem

# Reload to pick up any changes
importlib.reload(ou)
importlib.reload(s)
importlib.reload(disp)
importlib.reload(runs)

pd.set_option('display.max_columns', None)

## Load Models

In [2]:
# Load all models
model_configs = {
    "Mechanical Failure": config.MECH_FAIL_MODEL,
    "L4S1": config.L4S1_MODEL,
    "LL": config.LL_MODEL,
    "T4PA": config.T4PA_MODEL,
    "L1PA": config.L1PA_MODEL,
    "SVA": config.SVA_MODEL,
    "SS": config.SS_MODEL,
    "Global Tilt": config.GLOBAL_TILT_MODEL,
    "ODI": config.ODI_MODEL,
}

bundles = {name: ou.load_model_bundle(path) for name, path in model_configs.items()}

# Verify and display
print("Models loaded and verified:")
for name, bundle in bundles.items():
    status = "✓" if all(k in bundle for k in ["pipe", "features", "target"]) else "✗"
    print(f"  {status} {name}: {bundle.get('model_name', 'N/A')}")

# Extract individual bundles for use
mech_fail_bundle = bundles["Mechanical Failure"]
L4S1_bundle = bundles["L4S1"]
LL_bundle = bundles["LL"]
T4PA_bundle = bundles["T4PA"]
L1PA_bundle = bundles["L1PA"]
SVA_bundle = bundles["SVA"]
SS_bundle = bundles["SS"]
GT_bundle = bundles["Global Tilt"]
ODI_bundle = bundles["ODI"]

# Delta model bundles for postop predictions
delta_bundles = {
    "L4S1": L4S1_bundle,
    "LL": LL_bundle,
    "T4PA": T4PA_bundle,
    "L1PA": L1PA_bundle,
    "SS": SS_bundle,
    "GlobalTilt": GT_bundle,
    "SVA": SVA_bundle,
}

Models loaded and verified:
  ✓ Mechanical Failure: mech_fail_logreg
  ✓ L4S1: RidgeRegressor_delta_L4S1
  ✓ LL: RandomForest_delta_LL
  ✓ T4PA: RidgeRegressor_delta_T4PA
  ✓ L1PA: RidgeRegressor_delta_L1PA
  ✓ SVA: XGBRegressor_delta_SVA
  ✓ SS: XGBRegressor_delta_SS
  ✓ Global Tilt: XGBRegressor_delta_GlobalTilt
  ✓ ODI: RidgeRegressor_delta_ODI


In [3]:
# ---- Check categories in training data vs each model ----
train_df = pd.read_csv(config.DATA_PROCESSED)
print("Training data:", config.DATA_PROCESSED)
print(f"  Shape: {train_df.shape}")
print(f"  ASA_CLASS unique: {sorted(train_df['ASA_CLASS'].dropna().unique())}")
print(f"  sex unique: {sorted(train_df['sex'].dropna().unique())}")
print(f"  UIV_implant unique: {sorted(train_df['UIV_implant'].dropna().unique())}")
print(f"  num_levels_cat unique: {sorted(train_df['num_levels_cat'].dropna().unique())}")

Training data: /Users/vanjaglisic/Documents/UW_MSDS/CAPSTONE/repo work/Data-Science-Capstone/data/processed/cleaned_for_modeling.csv
  Shape: (271, 109)
  ASA_CLASS unique: ['1', '2', '3', '4', '4E']
  sex unique: ['FEMALE', 'MALE']
  UIV_implant unique: ['FS', 'Hook', 'PS']
  num_levels_cat unique: ['higher', 'lower']


## Optimization Scenarios

Each scenario uses a different set of composite score weights to explore how the optimizer behaves under different objectives. Predefined presets are in `src/runs.py`. To add a new scenario, add a new entry to `runs.PRESETS` and include its key below.

In [4]:
# =============================================================================
# SELECT SCENARIOS TO RUN
# =============================================================================
# Available presets (defined in src/runs.py):
#   "equal"           - All 6 alignment components weighted equally
#   "mech_fail"       - Primarily mech failure, small alignment weights
#   "mech_fail_t4l1pa"- Mech failure + T4PA-L1PA mismatch
#   "l4s1"            - Primarily L4-S1 in ideal range
#   "t4l1pa"          - Primarily T4PA-L1PA mismatch
#   "equal_plus_mech" - Composite (equal) blended 50/50 with mech failure
#   "odi"             - Primarily lowest predicted postop ODI
#   "gap_score"       - Primarily GAP score + category improvement
#   "ll"              - Primarily PI-LL mismatch
#
# To add a new scenario: add an entry to runs.PRESETS in src/runs.py,
# then include its key here.

SCENARIOS = ["equal", "mech_fail", "mech_fail_t4l1pa", "l4s1", "t4l1pa", "gap_score", "ll", "odi"]

# Show selected scenarios
for key in SCENARIOS:
    runs.print_preset(runs.PRESETS[key])


  Equal Weights (Composite)
  All 6 alignment components and mechanical failure weighted equally
                       GAP Score: 0.1429 ◀
                    L1PA penalty: 0.1429 ◀
                    L4S1 penalty: 0.1429 ◀
                  T4L1PA penalty: 0.1429 ◀
                      LL penalty: 0.1429 ◀
        GAP category improvement: 0.1429 ◀
         Mechanical failure prob: 0.1429 ◀

  Minimize Mechanical Failure
  Primarily mechanical failure probability, with small alignment weights
                       GAP Score: 0.0500 ◀
                    L1PA penalty: 0.0500 ◀
                    L4S1 penalty: 0.0500 ◀
                  T4L1PA penalty: 0.0500 ◀
                      LL penalty: 0.0500 ◀
        GAP category improvement: 0.0500 ◀
         Mechanical failure prob: 0.7000 ◀

  Minimize Mechanical Failure + T4L1PA
  Mechanical failure probability and T4PA-L1PA mismatch weighted equally
                       GAP Score: 0.0000
                    L1PA penalty: 0.0000
 

In [5]:
UIV_CHOICES, xl, xu = ou.get_decision_config()

print("UIV_CHOICES:", UIV_CHOICES)
print("xl:", xl)
print("xu:", xu)

# Use column names from config
print(pd.DataFrame([xl, xu], index=["xl","xu"], columns=config.DECISION_VAR_NAMES))

UIV_CHOICES: ['Hook', 'PS', 'FS']
xl: [0 0 0 0 0 0 1 2 0]
xu: [2 1 5 1 1 1 6 4 1]
    uiv_code  num_levels_cat_code  num_interbody_fusion_levels  ALIF  XLIF  \
xl         0                    0                            0     0     0   
xu         2                    1                            5     1     1   

    TLIF  num_rods  num_pelvic_screws  osteotomy  
xl     0         1                  2          0  
xu     1         6                  4          1  


## Test Patient w fixed parameters

In [6]:
# Show holdout patients summary
holdout_df = pd.read_csv(config.DATA_HOLDOUT)

summary_cols = [
    "id", "description", "revision", "mech_fail_last",
    "gap_score_preop", "gap_category", "gap_score_postop", "gap_category_postop",
    "UIV_implant", "num_levels_cat", "num_interbody_fusion_levels",
    "ALIF", "XLIF", "TLIF", "num_rods", "num_pelvic_screws", "osteotomy",
    "ODI_preop"
]
display(holdout_df[summary_cols].set_index("id"))

# Select patient
PATIENT_ID = 6380632
# PATIENT_ID = 2964021  # patient with ODI
patient_fixed = ou.load_patient_data(patient_id=PATIENT_ID, data_path=config.DATA_HOLDOUT)
print(f"\nSelected patient: {PATIENT_ID}")

,description,revision,mech_fail_last,gap_score_preop,gap_category,gap_score_postop,gap_category_postop,UIV_implant,num_levels_cat,num_interbody_fusion_levels,ALIF,XLIF,TLIF,num_rods,num_pelvic_screws,osteotomy,ODI_preop
id,,,,,,,,,,,,,,,,,
818588,mech failure - small GAP improvement;,0,1.0,12.0,SD,10.0,SD,PS,lower,1,0,0,1,3,2,0,NaN
1176294,mech failure - large GAP improvement;,0,1.0,10.0,SD,1.0,P,FS,lower,1,1,1,0,4,2,1,NaN
2964021,revision - GAP improvement 3 to 1,0,0.0,3.0,MD,1.0,P,PS,lower,2,1,0,0,3,3,0,15.0
6380632,large GAP imporovement,0,0.0,12.0,SD,1.0,P,Hook,higher,2,1,0,0,3,2,0,NaN



Selected patient: 6380632


## Run All Scenarios

For each selected scenario, runs the GA optimizer and collects results. 

**Constraints (all scenarios):** 
- If `num_interbody_fusion_levels > 0`, at least one fusion type (`ALIF`, `XLIF`, or `TLIF`) must be selected.
- If `ALIF=1` and `XLIF=0` and `TLIF=0`, then `num_interbody_fusion_levels` must be < 4.

In [7]:
# Run all selected scenarios
all_results = {}

for key in SCENARIOS:
    preset = runs.PRESETS[key]
    print(f"\n▶ Running: {preset['label']}...")

    result = runs.run_optimization(
        preset=preset,
        patient_fixed=patient_fixed,
        delta_bundles=delta_bundles,
        mech_fail_bundle=mech_fail_bundle,
        xl=xl,
        xu=xu,
        odi_bundle=ODI_bundle,
        pop_size=100,
        n_gen=20,
        seed=42,
        verbose=False,
        top_n=4,
        score_tolerance=2,
    )
    all_results[key] = result
    print(f"  ✓ Best composite: {result['best_result']['composite_score']:.2f}"
          f"  |  Mech fail: {result['best_result']['mech_fail_prob']*100:.1f}%")

print(f"\n{'='*60}")
print(f"Completed {len(all_results)} scenarios.")


▶ Running: Equal Weights (Composite)...
  ✓ Best composite: 3.35  |  Mech fail: 7.0%

▶ Running: Minimize Mechanical Failure...
  ✓ Best composite: 6.38  |  Mech fail: 7.9%

▶ Running: Minimize Mechanical Failure + T4L1PA...
  ✓ Best composite: 1.00  |  Mech fail: 2.0%

▶ Running: Minimize L4S1 Penalty...
  ✓ Best composite: 0.34  |  Mech fail: 1.7%

▶ Running: Minimize T4L1PA Penalty...
  ✓ Best composite: 0.98  |  Mech fail: 4.9%

▶ Running: Minimize GAP Score...
  ✓ Best composite: 8.96  |  Mech fail: 6.3%

▶ Running: Minimize LL (PI-LL) Penalty...
  ✓ Best composite: 1.27  |  Mech fail: 2.0%

▶ Running: Minimize Postop ODI...
  ✓ Best composite: 0.40  |  Mech fail: 2.0%

Completed 8 scenarios.


## Actual surgical plan & outcome for comparison

In [8]:
df_actual = disp.display_actual_outcomes(
    PATIENT_ID, patient_fixed, data_path=config.DATA_HOLDOUT,
    delta_bundles=delta_bundles,
    mech_fail_bundle=mech_fail_bundle,
    odi_bundle=ODI_bundle,
    presets=runs.PRESETS,
)
display(df_actual)

ACTUAL SURGICAL PLAN (WHAT WAS PERFORMED)
  UIV_implant: Hook
  num_levels_cat: higher
  num_interbody_fusion_levels: 2
  ALIF: 1
  XLIF: 0
  TLIF: 0
  num_rods: 3
  num_pelvic_screws: 2
  osteotomy (PSO): 0

  Mechanical Failure: No
  ODI: N/A (preop) → N/A (12mo)

MODEL SCORES FOR ACTUAL PLAN
  Mech Failure Prob (predicted): 42.7%

  Scenario Scores (lower is better):
    [equal]             Equal Weights (Composite)              16.01
    [mech_fail]         Minimize Mechanical Failure            33.35
    [mech_fail_t4l1pa]  Minimize Mechanical Failure + T4L1PA   21.34
    [l4s1]              Minimize L4S1 Penalty                   8.54
    [t4l1pa]            Minimize T4L1PA Penalty                 8.54
    [equal_plus_mech]   Equal Weights + Mechanical Failure     27.12
    [odi]               Minimize Postop ODI                     8.54
    [gap_score]         Minimize GAP Score                     36.77
    [ll]                Minimize LL (PI-LL) Penalty             9.26

ALIGN

,Parameter,Preop,Delta (actual),Postop (actual)
0,LL,30.4,32.5,62.9
1,SS,33.8,11.5,45.3
2,L4S1,34.5,7.0,41.5
3,GlobalTilt,42.2,-23.9,18.3
4,T4PA,24.8,-15.2,9.6
5,L1PA,12.2,-4.8,7.4
6,PI,58.6,2.2,60.8
7,PT,24.8,-9.3,15.5
8,SVA,154.6,-124.8,29.8
9,Age,76,-,-


## Results per Scenario

In [9]:
from IPython.display import display, Markdown

for key in SCENARIOS:
    r = all_results[key]

    display(Markdown(f"### {r['label']}"))
    display(Markdown(f"*{runs.PRESETS[key]['description']}*"))

    # # Best solution
    # df_comparison = disp.display_optimized_solution(r["best_result"], patient_fixed)
    # display(df_comparison)

    # Top 4 diverse plans
    display(Markdown("**Top diverse plans:**"))
    _ = disp.display_multiple_solutions(r["diverse_df"], patient_fixed)

### Equal Weights (Composite)

*All 6 alignment components and mechanical failure weighted equally*

**Top diverse plans:**

SOLUTIONS COMPARISON


,Parameter,Sol 1,Sol 2,Sol 3,Sol 4
0,Composite Score,1.46,1.49,2.74,1.5
1,Optimization Score,1.92,2.0,2.75,2.81
2,Mech Fail Prob,4.6%,5.1%,2.8%,10.7%
3,Predicted ODI,N/A (no preop ODI),N/A (no preop ODI),N/A (no preop ODI),N/A (no preop ODI)
4,GAP Score,12.0 (SD) → 1 (P),12.0 (SD) → 1 (P),12.0 (SD) → 2 (P),12.0 (SD) → 1 (P)
5,────────────,──────────,──────────,──────────,──────────
6,SURGICAL PLAN,,,,
7,UIV_implant,Hook,Hook,Hook,FS
8,num_levels_cat,lower,lower,lower,lower
9,num_interbody_fusion_levels,4,4,4,4


...

### Minimize Postop ODI

*Primarily lowest predicted postoperative ODI, with mech failure guard*

**Top diverse plans:**

SOLUTIONS COMPARISON


,Parameter,Sol 1,Sol 2,Sol 3,Sol 4
0,Composite Score,10.36,10.46,10.34,10.38
1,Optimization Score,0.27,0.55,0.66,0.96
2,Mech Fail Prob,1.4%,2.7%,3.3%,4.8%
3,Predicted ODI,N/A (no preop ODI),N/A (no preop ODI),N/A (no preop ODI),N/A (no preop ODI)
4,GAP Score,12.0 (SD) → 4 (MD),12.0 (SD) → 4 (MD),12.0 (SD) → 4 (MD),12.0 (SD) → 4 (MD)
5,────────────,──────────,──────────,──────────,──────────
6,SURGICAL PLAN,,,,
7,UIV_implant,Hook,PS,Hook,FS
8,num_levels_cat,lower,lower,lower,lower
9,num_interbody_fusion_levels,5,5,4,4
